In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

In [ ]:
%cd '/content/gdrive/My Drive/IDLSProject-main'

In [1]:
import pandas as pd
import os
import pickle

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_amazon.pkl'), "rb") as f:
    word_dict = pickle.load(f)

In [2]:
import pandas as pd
import os
train = pd.read_csv(os.path.join(data_dir, 'train_amazon.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_amazon.csv'), header=None, names=None)
print(train.shape, test_sample.shape)

(83000, 502) (21975, 502)


In [3]:
from sklearn.model_selection import train_test_split
test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

((83000, 502), (10987, 502), (10988, 502))

In [4]:
import torch
import torch.utils.data

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float().squeeze()
train_X = torch.from_numpy(train.drop([0], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float().squeeze()
val_X = torch.from_numpy(val.drop([0], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float().squeeze()
test_X = torch.from_numpy(test.drop([0], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10987])


In [5]:
import torch.nn as nn
import numpy as np

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, trial,vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()
        dropout = trial.suggest_uniform("dropout",0.1, 0.6)
        hidden_dim = trial.suggest_int("hidden_dim",16,256)
        embedding_dim = trial.suggest_int("embedding_dim",16,128)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig(out.squeeze())

In [13]:
filename = ""
def write_to_csv(trail_num, epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    global filename
    filename = "./lstm-amazon-hyperband-trails/"+str(trail_num)+".csv"
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)    
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [14]:
import time
import optuna
from optuna.trial import TrialState
import torch.optim as optim
import torch.optim as optim
import numpy as np

epochs = 10
vocab_size=10000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define an objective function to be minimized.
def objective(trial):
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    #epochs = trial.suggest_int("epochs",5,15)
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-0)
    #momentum = trial.suggest_uniform("momentum", 0.0, 1.0)
    model = LSTMClassifier(trial, vocab_size).to(device)
    trial.set_user_attr(key="best_model", value=model)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    loss_fn = torch.nn.BCELoss()
    train_loss_epoch = []
    train_acc_epoch = []
    val_loss_epoch = []
    val_accuracy_epoch = []
    time_train = []
    final_val_acc = 0
    for epoch in range(epochs):
        model.train()
        start = time.time()
        total_loss = 0
        train_acc = 0
        total = 0
        correct = 0
        for batch in train_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            optimizer.zero_grad()
            prediction = model(batch_X)
            loss = loss_fn(prediction, batch_y)
            loss.backward()
            optimizer.step()
            result = np.round(prediction.detach().cpu())
            total_loss += loss.data.item()
            total += batch_y.size(0)
            correct += (result == batch_y.cpu()).sum().item()
            train_acc = correct/total
        train_loss_epoch.append(np.round(total_loss / len(train_dl), 3))
        train_acc_epoch.append(np.round(train_acc*100,3))
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_dl)))
        with torch.no_grad():
            model.eval()
            correct = 0
            total = 0
            val_loss = []
            for inputs, labels in val_dl:
                inputs_val, labels_val = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                prediction = model(inputs_val)
                loss = loss_fn(prediction, labels_val)
                val_loss.append(np.round(loss.item(),3))
                result = np.round(prediction.cpu())
                total += labels_val.size(0)
                correct += (result == labels_val.cpu()).sum().item()
            val_accuracy_epoch.append(np.round((correct/total)*100, 3))
            val_loss_epoch.append(np.round(np.mean(val_loss),3))
            end = time.time() - start
            final_val_acc = np.round((correct/total)*100, 3)
            print("Val Loss: {:.3f}".format(np.mean(val_loss)), "\tVal Acc: {:.3f}".format(correct/total))
            time_train.append(np.round(end,3))
    write_to_csv(trial.number, epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_accuracy_epoch, time_train)
    return final_val_acc


In [15]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [ ]:
import optuna
from optuna.trial import TrialState
study = optuna.create_study(direction="maximize",pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=epochs, reduction_factor=3
    ),)
study.optimize(objective, n_trials=10,callbacks=[callback])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
best_model=study.user_attrs["best_model"]
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-14 13:09:34,436] A new study created in memory with name: no-name-c83de7c0-2522-4c8e-9c5e-b7327c12c223


Epoch: 0, BCELoss: 0.6929224205304341
Val Loss: 0.671 	Val Acc: 0.732
Epoch: 1, BCELoss: 0.6525878422590623
Val Loss: 0.637 	Val Acc: 0.759
Epoch: 2, BCELoss: 0.6233946706157133
Val Loss: 0.613 	Val Acc: 0.759
Epoch: 3, BCELoss: 0.602245904954083
Val Loss: 0.596 	Val Acc: 0.759
Epoch: 4, BCELoss: 0.5868950699467257
Val Loss: 0.583 	Val Acc: 0.759
Epoch: 5, BCELoss: 0.5757226680955255
Val Loss: 0.574 	Val Acc: 0.759
Epoch: 6, BCELoss: 0.5675644679959998
Val Loss: 0.568 	Val Acc: 0.759
Epoch: 7, BCELoss: 0.5615839402359653
Val Loss: 0.563 	Val Acc: 0.759
Epoch: 8, BCELoss: 0.5571794847587505
Val Loss: 0.559 	Val Acc: 0.759
Epoch: 9, BCELoss: 0.5539178863706359


[I 2022-05-14 13:13:09,073] Trial 0 finished with value: 75.937 and parameters: {'optimizer': 'SGD', 'lr': 0.0003415487497045099, 'dropout': 0.279376290360106, 'hidden_dim': 31, 'embedding_dim': 22}. Best is trial 0 with value: 75.937.


Val Loss: 0.557 	Val Acc: 0.759
Epoch: 0, BCELoss: 0.6698176515389638
Val Loss: 0.646 	Val Acc: 0.756
Epoch: 1, BCELoss: 0.6269680845091142
Val Loss: 0.613 	Val Acc: 0.759
Epoch: 2, BCELoss: 0.5995481013892645
Val Loss: 0.591 	Val Acc: 0.759
Epoch: 3, BCELoss: 0.5816590683467416
Val Loss: 0.578 	Val Acc: 0.759
Epoch: 4, BCELoss: 0.5697563487363149
Val Loss: 0.568 	Val Acc: 0.759
Epoch: 5, BCELoss: 0.5616765279008682
Val Loss: 0.562 	Val Acc: 0.759
Epoch: 6, BCELoss: 0.5560757879571743
Val Loss: 0.558 	Val Acc: 0.759
Epoch: 7, BCELoss: 0.5521045908511403
Val Loss: 0.554 	Val Acc: 0.759
Epoch: 8, BCELoss: 0.5492171682327627
Val Loss: 0.552 	Val Acc: 0.759
Epoch: 9, BCELoss: 0.5470576025635363


[I 2022-05-14 13:17:04,704] Trial 1 finished with value: 75.937 and parameters: {'optimizer': 'SGD', 'lr': 0.0004810573495938472, 'dropout': 0.23857225819264763, 'hidden_dim': 62, 'embedding_dim': 105}. Best is trial 0 with value: 75.937.


...

Val Loss: 0.288 	Val Acc: 0.878
Epoch: 0, BCELoss: 23.417496029188833
Val Loss: 23.917 	Val Acc: 0.759
Epoch: 1, BCELoss: 23.521950323610422
Val Loss: 24.009 	Val Acc: 0.759
Epoch: 2, BCELoss: 23.57173172766904
Val Loss: 24.055 	Val Acc: 0.759
Epoch: 3, BCELoss: 23.569122630429554
Val Loss: 24.062 	Val Acc: 0.759
Epoch: 4, BCELoss: 23.56867469879518
Val Loss: 24.062 	Val Acc: 0.759
Epoch: 5, BCELoss: 23.56867469879518
Val Loss: 24.062 	Val Acc: 0.759
Epoch: 6, BCELoss: 23.56867469879518
Val Loss: 24.062 	Val Acc: 0.759
Epoch: 7, BCELoss: 23.56867469879518
Val Loss: 24.062 	Val Acc: 0.759
Epoch: 8, BCELoss: 23.56867469879518
Val Loss: 24.062 	Val Acc: 0.759
Epoch: 9, BCELoss: 23.56867469879518


[I 2022-05-14 13:29:13,925] Trial 3 finished with value: 75.937 and parameters: {'optimizer': 'RMSprop', 'lr': 0.9774651686043574, 'dropout': 0.10488608806979502, 'hidden_dim': 101, 'embedding_dim': 36}. Best is trial 2 with value: 87.768.


Val Loss: 24.062 	Val Acc: 0.759
Epoch: 0, BCELoss: 0.688773082501917
Val Loss: 0.685 	Val Acc: 0.571
Epoch: 1, BCELoss: 0.6811577063008963
Val Loss: 0.678 	Val Acc: 0.614
Epoch: 2, BCELoss: 0.6739817524171737
Val Loss: 0.671 	Val Acc: 0.651
Epoch: 3, BCELoss: 0.6672177744917123
Val Loss: 0.665 	Val Acc: 0.686
Epoch: 4, BCELoss: 0.6608400853642498
Val Loss: 0.659 	Val Acc: 0.711
Epoch: 5, BCELoss: 0.6548248460852956
Val Loss: 0.653 	Val Acc: 0.732
Epoch: 6, BCELoss: 0.6491496205329895
Val Loss: 0.648 	Val Acc: 0.743
Epoch: 7, BCELoss: 0.643793596596603
Val Loss: 0.643 	Val Acc: 0.748
Epoch: 8, BCELoss: 0.6387373020131903
Val Loss: 0.638 	Val Acc: 0.753
Epoch: 9, BCELoss: 0.6339623708681887


[I 2022-05-14 13:32:48,834] Trial 4 finished with value: 75.564 and parameters: {'optimizer': 'SGD', 'lr': 5.929332656984242e-05, 'dropout': 0.2734163696086287, 'hidden_dim': 31, 'embedding_dim': 48}. Best is trial 2 with value: 87.768.


Val Loss: 0.633 	Val Acc: 0.756
Epoch: 0, BCELoss: 0.43107607761420397
Val Loss: 0.343 	Val Acc: 0.854
Epoch: 1, BCELoss: 0.33209891272147735
Val Loss: 0.313 	Val Acc: 0.864
Epoch: 2, BCELoss: 0.3027145309306412
Val Loss: 0.310 	Val Acc: 0.870


In [ ]:
torch.save(best_model.state_dict(),"./lstm-amazon-hyperband/lstm_amazon_hyperband.pth")

In [ ]:
def test(model, test_dl, epochs):
    model.eval()
    correct = 0
    total = 0
#     results = []
#     labels = []
    with torch.no_grad():
        for batch in test_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            prediction = model(batch_X)
            result = np.round(prediction.cpu())
#             results.extend(list(result.numpy()))
#             labels.extend(list(batch_y.numpy()))
            total += batch_y.size(0)
            correct += (result == batch_y).sum().item()
    print("Accuracy:", correct/total)

In [ ]:
test(best_model, test_dl, epochs)